# Acessando o banco de dados ChEMBL pelo *ChEMBL webresource client*

Para usar esse módulo, é necessário instalar o cliente utilizando:

`pip install chembl_webresource_client`

Tendo instalado o cliente, é necessário importá-lo, usando:

In [6]:
from chembl_webresource_client.new_client import new_client

In [20]:
import chembl_webresource_client
chembl_webresource_client.__version__

'0.10.8'

O [repositório no GitHub](https://github.com/chembl/chembl_webresource_client) sugere algumas formas de uso, como:

## Procurar uma estrutura (molecule) pelo nome

In [7]:
molecule = new_client.molecule
res = molecule.search('viagra')
type(res)

chembl_webresource_client.query_set.QuerySet

Veja que o resultado é um objeto do tipo QuerySet, contendo um extenso dicionário:

In [8]:
res

[{'atc_classifications': [], 'availability_type': -1, 'biotherapeutic': None, 'black_box_warning': 0, 'chebi_par_id': None, 'chemical_probe': 0, 'chirality': -1, 'cross_references': [], 'dosed_ingredient': False, 'first_approval': None, 'first_in_class': -1, 'helm_notation': None, 'indication_class': None, 'inorganic_flag': -1, 'max_phase': None, 'molecule_chembl_id': 'CHEMBL4792718', 'molecule_hierarchy': {'active_chembl_id': 'CHEMBL4792718', 'molecule_chembl_id': 'CHEMBL4792718', 'parent_chembl_id': 'CHEMBL4792718'}, 'molecule_properties': {'alogp': None, 'aromatic_rings': None, 'cx_logd': None, 'cx_logp': None, 'cx_most_apka': None, 'cx_most_bpka': None, 'full_molformula': 'C31H44N6O8S', 'full_mwt': '660.79', 'hba': None, 'hba_lipinski': None, 'hbd': None, 'hbd_lipinski': None, 'heavy_atoms': None, 'molecular_species': None, 'mw_freebase': '660.79', 'mw_monoisotopic': '660.2941', 'np_likeness_score': None, 'num_lipinski_ro5_violations': None, 'num_ro5_violations': None, 'psa': None,

In [9]:
# Chaves do dicionário
res[0].keys()

dict_keys(['atc_classifications', 'availability_type', 'biotherapeutic', 'black_box_warning', 'chebi_par_id', 'chemical_probe', 'chirality', 'cross_references', 'dosed_ingredient', 'first_approval', 'first_in_class', 'helm_notation', 'indication_class', 'inorganic_flag', 'max_phase', 'molecule_chembl_id', 'molecule_hierarchy', 'molecule_properties', 'molecule_structures', 'molecule_synonyms', 'molecule_type', 'natural_product', 'oral', 'parenteral', 'polymer_flag', 'pref_name', 'prodrug', 'score', 'structure_type', 'therapeutic_flag', 'topical', 'usan_stem', 'usan_stem_definition', 'usan_substem', 'usan_year', 'withdrawn_flag'])

In [10]:
print("Tipo de molécula:", res[0]['molecule_type'])
print("Número identificador no ChEMBL:", res[0]['molecule_chembl_id'])
print("É um medicamento tomado por via oral?", res[0]['oral'])
print("SMILES:", res[0]['molecule_structures']['canonical_smiles'])  # molecule_structures é um dict dentro de um dict

Tipo de molécula: Unknown
Número identificador no ChEMBL: CHEMBL4792718
É um medicamento tomado por via oral? False
SMILES: CC(=O)CC(O)(CC(C)=O)C(C)=O.CCCc1nn(C)c2c(=O)nc(-c3cc(S(=O)(=O)N4CCN(C)CC4)ccc3OCC)[nH]c12


## Procurar um alvo (*target*) pelo nome do gene

In [11]:
target = new_client.target
gene_name = 'BRD4'
res = target.search(gene_name)

In [12]:
res[0].keys()

dict_keys(['cross_references', 'organism', 'pref_name', 'score', 'species_group_flag', 'target_chembl_id', 'target_components', 'target_type', 'tax_id'])

## Encontre compostos similares à aspirina (similaridade > 70%)

In [13]:
molecule = new_client.molecule
similarity = new_client.similarity
aspirin_chembl_id = molecule.search('aspirin')[0]['molecule_chembl_id']
res = similarity.filter(chembl_id=aspirin_chembl_id, similarity=70)

In [14]:
for item in res:
    print(item['molecule_structures']['canonical_smiles'])

CC(=O)Oc1ccccc1C(=O)O.NCCCC[C@H](N)C(=O)O
CC(=O)Oc1ccccc1C(=O)O.NCCCCC(N)C(=O)O
CC(=O)Oc1ccccc1C(=O)O.CC(=O)Oc1ccccc1C(=O)O.NC(N)=O
CC(=O)Oc1ccccc1C(=O)[O-].CC(=O)Oc1ccccc1C(=O)[O-].NC(N)=O.[Ca+2]
CC(=O)Oc1ccccc1C(=O)Oc1ccccc1C(=O)O
O=C(O)Oc1ccccc1C(=O)O
CC(=O)Oc1cccc(C(=O)O)c1OC(C)=O


## Gerar um dataframe com todos os inibidores de um alvo

Escrevi uma função que recebe como entrada o código ChEMBL do alvo desejado. Para descobrir o código de um alvo, acesse o [site do ChEMBL](https://www.ebi.ac.uk/chembl/) e procure pelo alvo na barra *Search in ChEMBL*. A função retorna um DataFrame contendo as estruturas (SMILES) e atividades (pChEMBL) dos compostos

In [15]:
import pandas as pd

In [16]:
activities = new_client.activity

In [17]:
def get_inhibitors(target):
    """
    Returns a dataframe with SMILES and pChEMBL values of inhibitors for a given target
    Example: target = Cathepsin B = "CHEMBL4072"
    """
    inhibitors = activities.filter(target_chembl_id=target, pchembl_value__isnull=False)
    data = [[item["canonical_smiles"], item["pchembl_value"]] for item in inhibitors]
    df = pd.DataFrame(data, columns=["SMILES", "pChEMBL"])
    return df

In [18]:
# Encontrando inibidores da enzima Catepsina B
df = get_inhibitors("CHEMBL4072")  # código ChEMBL da Catepsina B
df.dropna(inplace=True)
df

,SMILES,pChEMBL
0,CC(C)C[C@H](C=O)NC(=O)[C@@H](NS(=O)(=O)c1ccc(F...,7.11
1,O=C(NC(=O)[C@H](Cc1cc(I)c(O)c(I)c1)NC(=O)[C@H]...,6.47
2,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(...,8.21
3,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H]1C(=O)...,4.52
4,O=C(N[C@@H](Cc1ccccc1)C(=O)N[C@@H]1C(=O)N2CCO[...,5.75
...,...,...
1446,CC[C@H](C)[C@@H](C(=O)O[C@@H](CC(C)C)C(=O)N[C@...,6.93
1447,COC1=CC(=O)N(C(=O)/C=C/[C@H](C)NC(=O)[C@H](CC(...,6.07
1448,COC1=CC(=O)N(C(=O)/C=C/[C@H](C)NC(=O)[C@H](CCc...,6.46
1449,COC1=CC(=O)N(C(=O)/C=C/[C@H](C)NC(=O)[C@H](CC(...,6.07


O download desse *dataset* demorou cerca de 4 minutos no meu computador. Seria mais rápido baixar pelo *site* do ChEMBL

Vamos ver abaixo uma forma diferente de baixar dados do ChEMBL que permite definir a versão que desejamos usar. As versões do ChEMBL são atualizadas periodicamente, com a inclusão de dados e correção de erros. [Aparentemente](https://github.com/chembl/chembl_webresource_client/issues/55) ainda não há como definir a versão usando o *ChEMBL webresource client*, mas há uma previsão de atualização para a versão 34.

# Acessando o banco de dados ChEMBL pelo chembl-downloader

Durante a User Group Meeting do RDKit (12th RDKit UGM, 2023), fiquei sabendo da existência do módulo `chembl-downloader`, que possibilita baixar *datasets* do ChEMBL definindo não somente os filtros desejados, mas também a versão do ChEMBL que desejamos usar. Com isso, podemos usar tanto a versão mais recente quanto uma versão antiga para fins de reprodução de um artigo, por exemplo.

O *ChEMBL webresource client*, que apresentei anteriormente neste Notebook, é bem lento para baixar bancos de dados grandes. Por isso, eu geralmente opto por baixá-los diretamente do site do ChEMBL. Infelizmente, o `chembl-downloader` parece sofrer do mesmo problema: para cada versão do ChEMBL que desejamos usar, ele precisa baixar o *database* do ChEMBL no formato SQL (o da versão 23, por exemplo, tem 2.55 GB). Porém, uma vez baixada a versão, você pode rapidamente acessar os dados desejados.

Baseei esse Notebook nos tutoriais disponibilizados pelo autor do módulo em https://github.com/cthoyt/chembl-downloader/tree/main/notebooks


In [ ]:
!pip install chembl-downloader

In [ ]:
import chembl_downloader

Importar todos os inibidores de um alvo para um DataFrame é simples:

In [ ]:
from chembl_downloader.contrib import get_target_smi_df

In [ ]:
# DataFrame com todos os compostos referentes ao alvo 5-lipoxygenase activating protein (CHEMBL4550).
df = get_target_smi_df("CHEMBL4550", version="23", aggregate=None).sort_values("molecule_chembl_id")

df.head()

Note o uso de aggregate=None para não agregar as duplicatas. Caso não seja declarado, elas serão agrupadas com a média aritmética.

Para usar a última versão:

In [ ]:
latest_version = chembl_downloader.latest()
print(latest_version)

In [ ]:
df = get_target_smi_df("CHEMBL4550", version=latest_version, aggregate=None).sort_values("molecule_chembl_id")

df.head()

O código a seguir demonstra o uso de outras *keywords* para selecionar os compostos com valores de IC<sub>50</sub> para o alvo definido em `target_chembl_id`:

In [ ]:
df = get_target_smi_df(
    target_chembl_id="CHEMBL4550",
    version=latest_version,
    aggregate=None,
    target_type="SINGLE PROTEIN",
    standard_relation="=",
    standard_type="IC50",
    tax_id="9606",  # Homo sapiens
)

df.head()

Também existem funcionalidades mais avançadas que envolvem o uso de funcionalidades da linguagem SQL e não vou abordar aqui.